In [1]:
import sys

sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

In [2]:
# Test Pareto Optimizer

import pandas as pd
import json
from typing import Dict, Any, List
import numpy as np
from datetime import datetime, timedelta
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs, Trial
from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer, ParetoData
from robyn.data.entities.enums import DependentVarType, PaidMediaSigns, OrganicSigns, ContextSigns

from utils.data_mapper import import_data, load_data_from_json

/Users/yijuilee/robynpy_release_reviews/robynvenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data from JSON
loaded_data = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/test_data.json"
)
imported_data = import_data(loaded_data)
model_outputs = imported_data["model_outputs"]
display((model_outputs.hyper_bound_ng))

,facebook_S_alphas,facebook_S_gammas,facebook_S_thetas,newsletter_alphas,newsletter_gammas,newsletter_thetas,ooh_S_alphas,ooh_S_gammas,ooh_S_thetas,print_S_alphas,print_S_gammas,print_S_thetas,search_S_alphas,search_S_gammas,search_S_thetas,tv_S_alphas,tv_S_gammas,tv_S_thetas,train_size,lambda
0,0.5,0.3,0.0,0.5,0.3,0.1,0.5,0.3,0.1,0.5,0.3,0.1,0.5,0.3,0.0,0.5,0.3,0.3,0.5,0
1,3.0,1.0,0.3,3.0,1.0,0.4,3.0,1.0,0.4,3.0,1.0,0.4,3.0,1.0,0.3,3.0,1.0,0.8,0.8,1


In [11]:
mmm_data = imported_data["mmm_data"]
display(mmm_data.data.head())
# Display Model Outputs

model_outputs = imported_data["model_outputs"]
display((model_outputs.trials[0].result_hyp_param))

hyperparameters = imported_data["hyperparameters"]
display(hyperparameters)

featurized_mmm_data = imported_data["featurized_mmm_data"]

holidays_data = imported_data["holidays_data"]

,DATE,revenue,tv_S,ooh_S,print_S,facebook_I,search_clicks_P,search_S,competitor_sales_B,facebook_S,events,newsletter
0,2015-11-23,2.754372e+06,22358.3467,0.0,12728.4889,2.430128e+07,0.0000,0.0000,8125009,7607.1329,na,19401.6538
1,2015-11-30,2.584277e+06,28613.4533,0.0,0.0000,5.527033e+06,9837.2385,4133.3333,7901549,1141.9525,na,14791.0000
2,2015-12-07,2.547387e+06,0.0000,132278.4,453.8667,1.665159e+07,12044.1197,3786.6667,8300197,4256.3754,na,14544.0000
3,2015-12-14,2.875220e+06,83450.3067,0.0,17680.0000,1.054977e+07,12268.0703,4253.3333,8122883,2800.4907,na,2800.0000
4,2015-12-21,2.215953e+06,0.0000,277336.0,0.0000,2.934090e+06,9467.2480,3613.3333,7105985,689.5826,na,15478.0000


,facebook_S_alphas,facebook_S_gammas,facebook_S_thetas,newsletter_alphas,newsletter_gammas,newsletter_thetas,ooh_S_alphas,ooh_S_gammas,ooh_S_thetas,print_S_alphas,...,lambda_hp,lambda_max,lambda_min_ratio,pos,Elapsed,ElapsedAccum,solID,trial,iterNG,iterPar
0,1.982029,0.584243,0.090727,1.751417,0.658512,0.228262,1.935087,0.804991,0.268471,1.347135,...,0.276251,6.548109e+07,0.0001,1,0.121680,0.113809,1_1_1,1,1,1
1,2.137653,0.423644,0.123685,1.721916,0.661215,0.278996,1.904210,0.597059,0.241335,1.468480,...,0.500997,6.548109e+07,0.0001,1,0.111287,0.111287,1_1_2,1,1,2
2,2.610267,0.796546,0.180935,1.474269,0.794208,0.218271,1.742035,0.579038,0.213408,2.664442,...,0.115435,6.548109e+07,0.0001,1,0.114888,0.118310,1_1_3,1,1,3
3,1.008753,0.477549,0.051473,0.972705,0.626001,0.249119,1.934297,0.545875,0.235703,1.432328,...,0.429544,6.548109e+07,0.0001,1,0.115740,0.122826,1_1_4,1,1,4
4,1.933208,0.629187,0.110968,1.857242,0.437809,0.199232,1.354886,0.744018,0.230233,1.667235,...,0.530087,6.548109e+07,0.0001,1,0.109141,0.121708,1_1_5,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,0.883798,0.532524,0.228413,1.509505,0.568951,0.214357,0.757741,0.767558,0.375191,2.084165,...,0.004555,6.548109e+07,0.0001,1,0.104000,23.955059,1_112_5,1,112,5
1004,1.621664,0.423359,0.283468,2.217514,0.911445,0.294430,0.554741,0.818780,0.384521,2.438405,...,0.011426,6.548109e+07,0.0001,1,0.120811,23.978861,1_112_6,1,112,6
1005,1.158597,0.573745,0.264026,1.465052,0.502724,0.192948,0.624082,0.800247,0.273677,0.931434,...,0.012699,6.548109e+07,0.0001,1,0.105688,23.969730,1_112_7,1,112,7
1006,1.414459,0.723852,0.213318,1.290304,0.718247,0.243058,0.918232,0.745184,0.257890,2.949644,...,0.004027,6.548109e+07,0.0001,1,0.109821,23.980340,1_112_8,1,112,8


Hyperparameters(hyperparameters={'facebook_S_alphas': [0.5, 3], 'facebook_S_gammas': [0.3, 1], 'facebook_S_thetas': [0, 0.3], 'print_S_alphas': [0.5, 3], 'print_S_gammas': [0.3, 1], 'print_S_thetas': [0.1, 0.4], 'tv_S_alphas': [0.5, 3], 'tv_S_gammas': [0.3, 1], 'tv_S_thetas': [0.3, 0.8], 'search_S_alphas': [0.5, 3], 'search_S_gammas': [0.3, 1], 'search_S_thetas': [0, 0.3], 'ooh_S_alphas': [0.5, 3], 'ooh_S_gammas': [0.3, 1], 'ooh_S_thetas': [0.1, 0.4], 'newsletter_alphas': [0.5, 3], 'newsletter_gammas': [0.3, 1], 'newsletter_thetas': [0.1, 0.4], 'train_size': [0.5, 0.8]}, adstock=<AdstockType.GEOMETRIC: 'geometric'>, lambda_=0.0, train_size=(0.5, 0.8), hyper_bound_list_updated={'facebook_S_alphas': [0.5, 3], 'facebook_S_gammas': [0.3, 1], 'facebook_S_thetas': [0, 0.3], 'print_S_alphas': [0.5, 3], 'print_S_gammas': [0.3, 1], 'print_S_thetas': [0.1, 0.4], 'tv_S_alphas': [0.5, 3], 'tv_S_gammas': [0.3, 1], 'tv_S_thetas': [0.3, 0.8], 'search_S_alphas': [0.5, 3], 'search_S_gammas': [0.3, 1], 

In [5]:
# 3. Create ParetoOptimizer instance
pareto_optimizer = ParetoOptimizer(mmm_data, model_outputs, hyperparameters, featurized_mmm_data, holidays_data)

In [6]:
# 4. Run optimize function
pareto_result = pareto_optimizer.optimize(pareto_fronts="auto", min_candidates=100)

>> Automatically selected [13.] Pareto-fronts  to contain at least 100 pareto-optimal models ([101])
>>> Calculating response curves for all models' media variables (505)...


/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py:453: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  pareto_fronts = int(auto_pareto["robynPareto"])


>> Pareto-Front: 1 [5 models]


Processing Solutions:   0%|          | 0/5 [00:00<?, ?solution/s]/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py:1207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_caov_pct.loc[:, df_caov_pct.columns[1:]] = df_caov_pct.loc[
Processing Solutions:  20%|██        | 1/5 [00:00<00:00,  9.72solution/s]/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py:1207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_caov_pct.loc[:, df_caov_pct.columns[1:]] = df_caov_pct.loc[
/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py

>> Pareto-Front: 2 [5 models]


...

Processing Solutions:   0%|          | 0/7 [00:00<?, ?solution/s]/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py:1207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_caov_pct.loc[:, df_caov_pct.columns[1:]] = df_caov_pct.loc[
/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py:1207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_caov_pct.loc[:, df_caov_pct.columns[1:]] = df_caov_pct.loc[
Processing Solutions:  29%|██▊       | 2/7 [00:00<00:00, 11.35solution/s]/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py

>> Pareto-Front: 12 [8 models]


Processing Solutions:   0%|          | 0/8 [00:00<?, ?solution/s]/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py:1207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_caov_pct.loc[:, df_caov_pct.columns[1:]] = df_caov_pct.loc[
/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py:1207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_caov_pct.loc[:, df_caov_pct.columns[1:]] = df_caov_pct.loc[
Processing Solutions:  25%|██▌       | 2/8 [00:00<00:00, 11.20solution/s]/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py

>> Pareto-Front: 13 [7 models]


Processing Solutions:   0%|          | 0/7 [00:00<?, ?solution/s]/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py:1207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_caov_pct.loc[:, df_caov_pct.columns[1:]] = df_caov_pct.loc[
/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py:1207: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_caov_pct.loc[:, df_caov_pct.columns[1:]] = df_caov_pct.loc[
Processing Solutions:  29%|██▊       | 2/7 [00:00<00:00, 11.06solution/s]/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/robyn/modeling/pareto/pareto_optimizer.py

In [12]:
# 5. Check results
print("Pareto Optimization Results:")
print(f"Number of Pareto fronts: {len(pareto_result.pareto_solutions)}")
print(f"MediaVecCollect: {pareto_result.media_vec_collect.shape, pareto_result.media_vec_collect}")
print("\Hyper parameter solutions:")
print(pareto_result.result_hyp_param)

print("\nAggregated decomposition results:")
print(pareto_result.x_decomp_agg)
print("\result Calibration:")
print(pareto_result.result_calibration)
print("\nx Decomp Vec Collect:")
print(pareto_result.x_decomp_vec_collect.shape, pareto_result.x_decomp_vec_collect)
print("\nCarryover percentage all:")
print(pareto_result.df_caov_pct_all.shape, pareto_result.df_caov_pct_all)
print("\Plot Data Collected")
print("NUMBER OF PLOTS Data collected for:", len(pareto_result.plot_data_collect["2_85_2"]))
print("Plot data for solid 2_85_2", pareto_result.plot_data_collect["2_85_2"])

# 6. Validate logic
assert pareto_result.pareto_fronts == "auto" or isinstance(
    pareto_result.pareto_fronts, int
), "Invalid pareto_fronts value"
assert not pareto_result.result_hyp_param.empty, "Empty result_hyp_param DataFrame"
assert not pareto_result.x_decomp_agg.empty, "Empty x_decomp_agg DataFrame"

print("\nAll assertions passed. The optimize function is working as expected.")

Pareto Optimization Results:
Number of Pareto fronts: 1
MediaVecCollect: ((1248, 9),               ds           tv_S          ooh_S       print_S     facebook_S  \
0     2015-11-23   22358.346700       0.000000  12728.488900    7607.132900   
1     2015-11-30   28613.453300       0.000000      0.000000    1141.952500   
2     2015-12-07       0.000000  132278.400000    453.866700    4256.375400   
3     2015-12-14   83450.306700       0.000000  17680.000000    2800.490700   
4     2015-12-21       0.000000  277336.000000      0.000000     689.582600   
...          ...            ...            ...           ...            ...   
1243  2018-12-03    4018.608260   71975.013722   8570.369851   63642.586143   
1244  2018-12-10     264.098672   41393.650579  19814.796993   27307.341849   
1245  2018-12-17      17.238214   42134.325828   2169.225589   10606.290973   
1246  2018-12-24  298161.263904   22919.124450  21123.762720  131306.133210   
1247  2018-12-31  168253.861660   12046.685283

## Allocator

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Any

from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs
from robyn.data.entities.hyperparameters import Hyperparameters
from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer, ParetoData
from robyn.allocator.entities.enums import OptimizationScenario, ConstrMode
from robyn.allocator.budget_allocator import BudgetAllocator
from robyn.allocator.entities.allocation_config import AllocationConfig
from robyn.allocator.entities.allocation_constraints import AllocationConstraints

print("Step 1: Prepare Model Data")
print("------------------------")

# Aggregate trial data into all_x_decomp_agg
all_decomp_list = []
for trial in model_outputs.trials:
    all_decomp_list.append(trial.x_decomp_agg)

# Update model_outputs with aggregated data
model_outputs.all_x_decomp_agg = pd.concat(all_decomp_list, axis=0)

print(f"Data loaded successfully:")
print(
    f"- Data timeframe: {mmm_data.data[mmm_data.mmmdata_spec.date_var].min()} to {mmm_data.data[mmm_data.mmmdata_spec.date_var].max()}"
)
print(f"- Number of paid media channels: {len(mmm_data.mmmdata_spec.paid_media_spends)}")
print(f"- Channels: {mmm_data.mmmdata_spec.paid_media_spends}")

print("\nStep 2: Setup Budget Allocator")
print("------------------------------")

# Get available models from aggregated data
available_models = model_outputs.all_x_decomp_agg["solID"].unique()
print(f"Available model IDs: {available_models}")

# Select first available model or specific one
select_model = model_outputs.select_id if model_outputs.select_id else available_models[0]
print(f"Using model ID: {select_model}")

# Initialize budget allocator
try:
    allocator = BudgetAllocator(
        mmm_data=mmm_data, model_outputs=model_outputs, hyperparameter=hyperparameters, select_model=select_model
    )
    print(f"Successfully initialized allocator for model {select_model}")
except Exception as e:
    print(f"Error initializing allocator: {str(e)}")
    print("Available model IDs:", available_models)
    raise e

# Rest of your notebook remains the same...
print("\nStep 3: Configure Allocation")
print("---------------------------")

# Set channel constraints
channel_constraints_low = {channel: 0.7 for channel in mmm_data.mmmdata_spec.paid_media_spends}
channel_constraints_up = {channel: 1.2 for channel in mmm_data.mmmdata_spec.paid_media_spends}

# Create constraints object
constraints = AllocationConstraints(
    channel_constr_low=channel_constraints_low,
    channel_constr_up=channel_constraints_up,
    channel_constr_multiplier=3.0,  # optional, will default to 3.0 if not provided
)

# Create allocation config
max_response_config = AllocationConfig(
    scenario=OptimizationScenario.MAX_RESPONSE,
    total_budget=5000000,  # Example budget
    target_value=None,  # Not needed for max_response scenario
    date_range="last_10",
    constraints=constraints,
    maxeval=100000,  # optional
    optim_algo="SLSQP_AUGLAG",  # optional
    constr_mode=ConstrMode.EQUALITY,  # optional
    plots=True,
    export=True,
    quiet=False,
)

print("Max Response Configuration:")
print(f"- Total Budget: ${max_response_config.total_budget:,.0f}")
print(f"- Date Range: {max_response_config.date_range}")
print(f"- Optimization Algorithm: {max_response_config.optim_algo}")
print(f"- Constraint Mode: {max_response_config.constr_mode}")

print("\nChannel Constraints:")
for channel in mmm_data.mmmdata_spec.paid_media_spends:
    print(
        f"- {channel}: {constraints.channel_constr_low[channel]:.1f}x to {constraints.channel_constr_up[channel]:.1f}x"
    )

print("\nStep 4: Run Allocation Optimization")
print("---------------------------------")

# Run allocation
result = allocator.allocate(max_response_config)

# Display results
print("\nOptimization Results:")
print("--------------------")
print(f"Total Spend: ${result.metrics['total_spend']:,.0f}")
print(f"Total Response: {result.metrics['total_response']:,.0f}")
print(f"Response Lift: {result.metrics['response_lift']*100:.1f}%")
print("\nOptimal Channel Allocations:")

allocations_df = result.optimal_allocations
for channel in mmm_data.mmmdata_spec.paid_media_spends:
    current = allocations_df.loc[allocations_df["channel"] == channel, "current_spend"].iloc[0]
    optimal = allocations_df.loc[allocations_df["channel"] == channel, "optimal_spend"].iloc[0]
    change = (optimal / current - 1) * 100

    print(f"{channel:>10}: ${current:,.0f} -> ${optimal:,.0f} ({change:+.1f}%)")

print("\nStep 5: Target Efficiency Scenario")
print("--------------------------------")

# Configure target efficiency scenario
target_config = AllocationConfig(
    scenario=OptimizationScenario.TARGET_EFFICIENCY,
    total_budget=None,  # Will be determined by optimization
    target_value=2.0,  # Target ROAS/CPA
    date_range="last_10",
    constraints=constraints,
    constr_mode=ConstrMode.EQUALITY,
    plots=True,
    export=True,
    quiet=False,
)

# Run target efficiency optimization
target_result = allocator.allocate(target_config)

print("\nTarget Efficiency Results:")
print("-------------------------")
print(f"Total Spend: ${target_result.metrics['total_spend']:,.0f}")
print(f"Total Response: ${target_result.metrics['total_response']:,.0f}")
print(f"Achieved Efficiency: {target_result.metrics['efficiency']:.2f}")

print("\nStep 6: Available Visualizations")
print("------------------------------")

if result.plots or target_result.plots:
    print("\nAvailable Plots:")
    for plot_type, plot_data in result.plots.items():
        print(f"- {plot_type}")

print("\nDemo Complete")

Step 1: Prepare Model Data
------------------------
Data loaded successfully:
- Data timeframe: DATE    2015-11-23
dtype: object to DATE    2019-11-11
dtype: object
- Number of paid media channels: 5
- Channels: ['tv_S', 'ooh_S', 'print_S', 'facebook_S', 'search_S']

Step 2: Setup Budget Allocator
------------------------------
Available model IDs: ['1_1_1' '1_1_2' '1_1_3' ... '2_112_7' '2_112_8' '2_112_9']
Using model ID: 1_1_1
Error initializing allocator: __init__() got an unexpected keyword argument 'mmm_data'
Available model IDs: ['1_1_1' '1_1_2' '1_1_3' ... '2_112_7' '2_112_8' '2_112_9']


TypeError: __init__() got an unexpected keyword argument 'mmm_data'

In [ ]:
from typing import Optional

print("Step 1: Debug ModelOutputs Data")
print("------------------------------")

# Check ModelOutputs contents
print("\nModelOutputs Structure:")
print(f"Number of trials: {len(model_outputs.trials)}")
print(f"Selected model ID: {model_outputs.select_id}")

# Check first trial data
if model_outputs.trials:
    first_trial = model_outputs.trials[0]
    print("\nFirst Trial Data:")
    print(f"Solution ID: {first_trial.sol_id}")
    print("\nx_decomp_agg columns:")
    print(first_trial.x_decomp_agg.columns.tolist())
    print("\nx_decomp_agg first few rows:")
    print(first_trial.x_decomp_agg.head())

# Check aggregated data
print("\nAggregated Data Status:")
print(f"all_x_decomp_agg empty? {model_outputs.all_x_decomp_agg.empty}")
if not model_outputs.all_x_decomp_agg.empty:
    print("all_x_decomp_agg columns:")
    print(model_outputs.all_x_decomp_agg.columns.tolist())
    print("\nFirst few rows:")
    print(model_outputs.all_x_decomp_agg.head())

# Let's try to aggregate the data if it's not already done
print("\nAttempting to aggregate trial data...")
try:
    # Combine x_decomp_agg from all trials
    all_decomp = pd.concat([trial.x_decomp_agg for trial in model_outputs.trials])
    print("\nManually aggregated data:")
    print(f"Shape: {all_decomp.shape}")
    print("Columns:", all_decomp.columns.tolist())
    if "solID" not in all_decomp.columns:
        print("\nNOTE: 'solID' column missing from trial data")
        # Check what might be the identifier column
        print("Available columns that might be identifiers:")
        id_columns = [
            col for col in all_decomp.columns if any(x in col.lower() for x in ["id", "solution", "sol", "model"])
        ]
        print(id_columns)
except Exception as e:
    print(f"\nError aggregating trial data: {str(e)}")

print("\nStep 2: Attempting to find model identifier")
print("----------------------------------------")

# Try different potential column names
potential_id_columns = ["solID", "sol_id", "solution_id", "model_id", "trial_id"]


def check_column_in_df(df: pd.DataFrame, column_names: List[str]) -> Optional[str]:
    """Check which of the potential column names exists in the DataFrame."""
    for col in column_names:
        if col in df.columns:
            return col
    return None


for trial in model_outputs.trials:
    id_column = check_column_in_df(trial.x_decomp_agg, potential_id_columns)
    if id_column:
        print(f"Found ID column in trial data: {id_column}")
        print(f"Unique IDs: {trial.x_decomp_agg[id_column].unique()}")
        break

# Print full trial structure of first trial
if model_outputs.trials:
    print("\nComplete structure of first trial:")
    trial = model_outputs.trials[0]
    print(f"Trial attributes:")
    for attr in vars(trial):
        if isinstance(getattr(trial, attr), pd.DataFrame):
            df = getattr(trial, attr)
            print(f"\n{attr}:")
            print(f"Shape: {df.shape}")
            print(f"Columns: {df.columns.tolist()}")
            print(f"First row: {df.iloc[0].to_dict()}")